# Get Dataset

In [7]:
# from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!unzip "path-to-zip" -d "/content/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000926.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000936.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000993.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000994.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000278.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000499.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000102.png  
  inflating: /content/TestSet/latent-diffusion_noise2image_FFHQ/latent-diffusion_noise2img-ffhq256_sample_000150.png  
  inflating:

In [22]:
FAKE_IMG_PATH = '/content/TestSet/'